(concepts:parsers)=
# Parsers

Parsers convert strings to values. To create a parser for a new type, you can simply subclass
the {class}`configpile.parsers.Parser` class, not forgetting that this class takes a type
parameter corresponding to the type of the value it parses.

In [1]:
from configpile import *

class FloatParser(Parser[float]):
    def parse(self, arg: str) -> Res[float]:
        try:
            return float(arg)
        except ValueError as e:
            return Err.make(str(e))

fp = FloatParser()


In [2]:
fp.parse("2")

2.0

In [3]:
fp.parse("invalid")

Err1(msg="could not convert string to float: 'invalid'", contexts=[])

## Parser construction

A parser can also be constructed in several ways.

### From multiple (string) choices

If the multiple choices are strings, use {meth}`~configpile.parsers.Parser.from_choices`. In
particular, one can force lower or upper case (which we do here).

In [4]:
cp = Parser.from_choices(["red", "green", "blue"], force_case= ForceCase.LOWER)

cp.parse("yellow")

Err1(msg='Value yellow not in choices red,green,blue', contexts=[])

In [5]:
cp.parse("blue")

'blue'

### From multiple choices corresponding to values

When the strings correspond to values, use {meth}`~configpile.parsers.Parser.from_mapping`.
The ``aliases`` are alternate strings, they are accepted during the parse but not showed in the
usage description.

In [6]:
bp = Parser.from_mapping({"true": True, "false": False},force_case=ForceCase.LOWER, aliases={"0": False, "1": True})

### From functions that raise exceptions

In this case, the exceptions will be converted to {class}`configpile.userr.Err` instances; 
one can specify which exceptions should be caught, and the rest will be propagated.

The static method {meth}`~configpile.parsers.Parser.from_function_that_raises` is useful for Python
code coming from standard or external libraries.

In [7]:
fp1 = Parser.from_function_that_raises(float, ValueError)

In [8]:
fp1.parse("2")

2.0

In [9]:
fp1.parse("invalid")

Err1(msg="Error 'could not convert string to float: 'invalid'' in 'invalid'", contexts=[])

### From functions that return a result

One does not need the big ceremonial of constructing a {class}`configpile.parsers.Parser` instance
as only one method needs to be implemented.

The static method {meth}`configpile.parsers.Parser.from_function` is useful for functions that
you will implement yourself: those functions take a string and return a 
{data}`configpile.userr.Res`, as discussed in {ref}`concepts:errors`.

In [10]:
def parse_fun(arg: str) -> Res[float]:
    try:
        return float(arg)
    except ValueError as e:
        return Err.make(str(e))

fp2 = Parser.from_function(parse_fun)

In [11]:
fp2.parse("2")

2.0

In [12]:
fp2.parse("invalid")

Err1(msg="could not convert string to float: 'invalid'", contexts=[])

### From [parsy](https://github.com/python-parsy/parsy) parsers

`parsy` is a parser combinator library, useful to parse more involved expressions.

Here is how you would use this library. Note that `configpile` has `parsy` as an optional
dependency, so that every `configpile` user is not forced to have `parsy` as a dependency.

Note that `parsy` does not support [type hints](https://peps.python.org/pep-0484/), so one needs
to provide the type as a first argument to {meth}`~configpile.parsers.Parser.from_parsy_parser`.

In [13]:
import parsy
int_parsy_parser = parsy.regex('[0-9]+').map(int)
ip: Parser[int] = Parser.from_parsy_parser(int_parsy_parser)
ip.parse("1234")

1234

## Parser manipulation

Parsers can be adapted as to change their return value, or even modify the parsing behavior.

### Handling optional parameters

Here, we mean optional in the sense of possibly having a {data}`None` value. We take, for example,
our {class}`float` parser, and make it optional.

In [14]:
from typing import Optional
fp_opt: Parser[Optional[float]] = fp.empty_means_none()

fp_opt.parse("") is None

True

In [15]:
fp_opt.parse("3.14")

3.14

### Handling sequences

One can parse sequences of values. Note that the pattern we show below is very crude, and
appropriate only when the separator is never part of the strings representing the values.

In [16]:
fp_seq = fp.separated_by(",")
fp_seq.parse("1,2,3,4")

[1.0, 2.0, 3.0, 4.0]

### Transforming the parser output

One can map the result of a parser through a function.

In [17]:
rounded_fp: Parser[int] = fp.map(int)
rounded_fp.parse("1.23")

1

## Validation

See {ref}`concepts:validation`.